In [1]:
''' Author - Danish Gufran
           - Jadon Johnson
           - Isaac Blair
           - Sudeep Pasricha

EPIC Lab : Colorado State University, Fort Collins

<Danish.Gufran@colostate.edu>
<Jadon.Johnson@colostate.edu>
<Isaac.Blair@colostate.edu>
<Sudeep@colostate.edu>

Title - Data Augmentation Methods for Wi-Fi RSS Fingerprinting Based Indoor Localization.

Citation :

'''

' Author - Danish Gufran\n           - Jadon Johnson\n           - Isaac Blair\n           - Sudeep Pasricha\n<Danish.Gufran@colostate.edu>\n<Jadon.Johnson@colostate.edu>\n<isaac.blair@colostate.edu>\n<sudeep@colostate.edu>\n\nData Augmentation Methods for Wi-Fi RSS Fingerprinting Based Indoor Localization.\n\nCitation : \n\n'

In [2]:
# !rm -rf maril
try:
  !git clone https://github.com/danishgufran/RSS_Database.git
  !git clone https://github.com/danishgufran/EPIC_Lab_Data.git
  !git clone https://github.com/EPIC-CSU/heterogeneous-rssi-indoor-nav.git
  !pip install tensorflow-addons
  !pip install keras-multi-head
  !pip install catboost

except:
  from git import Repo  # pip install gitpython
  Repo.clone_from("https://github.com/danishgufran/RSS_Database.git")
  Repo.clone_from("https://github.com/danishgufran/EPIC_Lab_Data.git")
  Repo.clone_from("https://github.com/EPIC-CSU/heterogeneous-rssi-indoor-nav.git")

fatal: destination path 'RSS_Database' already exists and is not an empty directory.
fatal: destination path 'EPIC_Lab_Data' already exists and is not an empty directory.
fatal: destination path 'heterogeneous-rssi-indoor-nav' already exists and is not an empty directory.


In [3]:
import numpy as np

import copy
from copy import deepcopy
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import *
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv1D, MaxPooling1D , LSTM, Attention
from tensorflow.keras.losses import *
from tensorflow.keras.optimizers import*
import random as random
import time
import matplotlib.pyplot as plt
import seaborn as sb
import pandas as pd

import RSS_Database.Stone_Seth.Seth
from RSS_Database.Stone_Seth.Seth import fetch_seth, Devices, Floorplan, get_mac_ids

from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from EPIC_Lab_Data.data import Devices, Floorplan, build_dataset
from EPIC_Lab_Data.helpers import compute_distances
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt

from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import lightgbm as lgb

import logging
import json
import numpy as np
import os
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.layers import LayerNormalization
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LayerNormalization, MultiHeadAttention, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.losses import Loss
from tensorflow.keras.optimizers import Adam
from sklearn import svm
import xgboost as xgb

from EPIC_Lab_Data.helpers import split_frame, compute_distances
from EPIC_Lab_Data.data import build_dataset
from EPIC_Lab_Data.Maril.MultiHeadAttentionAddon import MultiHeadAttentionAddon

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [4]:
def train_data(itr,dev, floorplan):
    # dfs is a list of dataframes
# meta is a dataframe with meta data

#getting train data

    train_fp, train_meta = fetch_seth(
    dev,
    str(floorplan),
    ci = int(itr),
    base_path="RSS_Database/Stone_Seth/temp/clean/"  # <-- this would be 'seth/temp/clean' from outside this dir
)
    # train_fp, _, macs, lbl2cord = build_dataset(
    #     dev,
    #     str(floorplan),
    # )
    train_fp = train_fp.sample(frac=1).reset_index(drop=True)
    train_aps = get_mac_ids(train_fp.columns)
    train_x = train_fp[train_aps].values
    # train_x = (train_x + 100)/100
    train_y = (train_fp["label"]).values
    return train_x, train_y, train_aps

def test_data(itr, train_aps, dev, floorplan):
    #getting test data
    test_fp, test_meta = fetch_seth(
    dev ,
    str(floorplan),
    ci = itr,
    base_path="RSS_Database/Stone_Seth/temp/clean/"  # <-- this would be 'seth/temp/clean' from outside this dir
)
    # train_df, test_fp, macs_test, lbl2cords = build_dataset(
    #       dev,
    #       str(floorplan)
    #   )
    test_y = test_fp["label"].values
    # train_aps = train_aps.drop(['x', 'y','label'], axis=1)
    # print(f'label -- {test_y}')
    test_aps = get_mac_ids(test_fp.columns)
    missing_aps = list(set(train_aps.columns)-set(test_aps))
    test_fp[missing_aps] = 0

    test_fp = test_fp.drop(['x', 'y','label'], axis=1)
    test_x = test_fp[:]

    # test_x = (test_x + 100)/100


    return test_x, test_y

def temp_train_data(dev, floorplan, ci_val):
    # dfs is a list of dataframes
# meta is a dataframe with meta data

#getting train data

    train_fp, train_meta = fetch_seth(
    dev,
    str(floorplan),
    ci = ci_val,
    base_path="RSS_Database/Stone_Seth/temp/clean/"  # <-- this would be 'seth/temp/clean' from outside this dir
)
    # train_fp, _, macs, lbl2cord = build_dataset(
    #     dev,
    #     str(floorplan),
    # )
    train_fp = train_fp.sample(frac=1).reset_index(drop=True)
    train_aps = get_mac_ids(train_fp.columns)
    train_x = train_fp[train_aps].values
    train_x = (train_x + 100)/100
    train_y = (train_fp["label"]).values
    return train_x, train_y, train_aps
def temp_test_data(train_aps, dev, floorplan, ci_val):
    #getting test data
    test_fp, test_meta = fetch_seth(
    str(dev) ,
    str(floorplan),
    ci = ci_val,
    base_path="RSS_Database/Stone_Seth/temp/clean/"  # <-- this would be 'seth/temp/clean' from outside this dir
)
    # train_df, test_fp, macs_test, lbl2cords = build_dataset(
    #       dev,
    #       str(floorplan)
    #   )
    test_aps = get_mac_ids(test_fp.columns)
    missing_aps = list(set(train_aps)-set(test_aps))
    test_fp[missing_aps] = 0
    test_x = test_fp[train_aps].values
    test_x = (test_x + 100)/100
    test_y = (test_fp["label"]).values
    return test_x, test_y

In [8]:
def stacked_sae(input, train_x, train_y, eph):
  print("\n Training Stacked SAE... \n")
  h1 = int(input - input*0.3)
  h2 = int(h1 - h1*0.3)
  h3 = int(h2 - h2*0.3)
  h4 = int(h3 - h3*0.3)
  #print(input,h1,h2,h3, h4)
  # AE -1
  input_img = keras.layers.Input(shape = (input, ))
  GN = keras.layers.GaussianNoise(0.)(input_img)
  distorted_input1 = Dropout(.1)(GN)
  encoded1 = Dense(h1, activation = 'gelu')(distorted_input1)
  decoded1 = Dense(input, activation = 'sigmoid')(encoded1)

  autoencoder1 = keras.models.Model(inputs = input_img, outputs = decoded1)
  encoder1 = keras.models.Model(inputs = input_img, outputs = encoded1)

  # AE -2
  encoded1_input = keras.layers.Input(shape = (h1,))
  GN1 = keras.layers.GaussianNoise(0.)(encoded1_input)
  distorted_input2 = Dropout(.10)(GN1)
  encoded2 = Dense(h2, activation = 'gelu')(distorted_input2)
  decoded2 = Dense(h1, activation = 'sigmoid')(encoded2)

  autoencoder2 = tf.keras.Model(inputs = encoded1_input, outputs = decoded2)
  encoder2 = tf.keras.Model(inputs = encoded1_input, outputs = encoded2)

  # AE -3
  encoded2_input = keras.layers.Input(shape = (h2,))
  GN2 = keras.layers.GaussianNoise(0.)(encoded2_input)
  distorted_input3 = Dropout(.10)(GN2)
  encoded3 = Dense(h3, activation = 'gelu')(distorted_input3)
  decoded3 = Dense(h2, activation = 'sigmoid')(encoded3)

  autoencoder3 = tf.keras.Model(inputs = encoded2_input, outputs = decoded3)
  encoder3 = tf.keras.Model(inputs = encoded2_input, outputs = encoded3)

  # AE -4
  encoded3_input = keras.layers.Input(shape = (h3,))
  GN3 = keras.layers.GaussianNoise(0.)(encoded3_input)
  distorted_input4 = Dropout(.10)(GN3)
  encoded4 = Dense(h4, activation = 'gelu')(distorted_input4)
  decoded4 = Dense(h3, activation = 'sigmoid')(encoded4)

  autoencoder4 = tf.keras.Model(inputs = encoded3_input, outputs = decoded4)
  encoder4 = tf.keras.Model(inputs = encoded3_input, outputs = encoded4)
  # Final AE
  encoded1_da = Dense(h1, activation = 'sigmoid')(input_img)
  encoded2_da = Dense(h2, activation = 'sigmoid')(encoded1_da)
  encoded3_da = Dense(h3, activation = 'sigmoid')(encoded2_da)
  encoded4_da = Dense(h4, activation = 'sigmoid')(encoded3_da)
  decoded4_da = Dense(h3, activation = 'sigmoid')(encoded4_da)
  decoded3_da = Dense(h2, activation = 'sigmoid')(decoded4_da)
  decoded2_da = Dense(h1, activation = 'sigmoid')(decoded3_da)
  decoded1_da = Dense(input, activation = 'sigmoid')(decoded2_da)

  deep_autoencoder = tf.keras.Model(inputs = input_img, outputs = decoded1_da)
  #compile
  sgd1 = tf.keras.optimizers.Adam(learning_rate = 0.001)
  sgd2 = tf.keras.optimizers.Adam(learning_rate = 0.001)
  sgd3 = tf.keras.optimizers.Adam(learning_rate = 0.001)
  sgd4 = tf.keras.optimizers.Adam(learning_rate = 0.001)

  sgdD = tf.keras.optimizers.Adam(learning_rate = 0.001)

  autoencoder1.compile(loss='mse', optimizer = sgd1)
  autoencoder2.compile(loss='mse', optimizer = sgd2)
  autoencoder3.compile(loss='mse', optimizer = sgd3)
  autoencoder4.compile(loss='mse', optimizer = sgd4)

  encoder1.compile(loss='mse', optimizer = sgd1)
  encoder2.compile(loss='mse', optimizer = sgd1)
  encoder3.compile(loss='mse', optimizer = sgd1)
  encoder4.compile(loss='mse', optimizer = sgd1)

  deep_autoencoder.compile(loss='mse', optimizer = sgdD)
  # fit ae 1
  autoencoder1.fit(train_x, train_x,
                epochs = eph,
                validation_split = 0.0020,
                shuffle = False, verbose =0,
                callbacks=[
                keras.callbacks.EarlyStopping(patience=200, restore_best_weights=True)])
  first_layer_code = encoder1.predict(train_x)
  #fit ae 2
  autoencoder2.fit(first_layer_code, first_layer_code,
                epochs = eph,
                validation_split = 0.0020,
                shuffle = False, verbose =0,
                callbacks=[
                keras.callbacks.EarlyStopping(patience=200, restore_best_weights=True)])
  second_layer_code = encoder2.predict(first_layer_code)
  #fit ae 3
  autoencoder3.fit(second_layer_code, second_layer_code,
               epochs = eph,
               validation_split = 0.0020,
               shuffle = False, verbose =0,
                callbacks=[
                keras.callbacks.EarlyStopping(patience=200, restore_best_weights=True)])
  third_layer_code = encoder3.predict(second_layer_code)
    #fit ae 4
  autoencoder4.fit(third_layer_code, third_layer_code,
               epochs = eph,
               validation_split = 0.0020,
               shuffle = False, verbose =0,
                callbacks=[
                keras.callbacks.EarlyStopping(patience=2000, restore_best_weights=True)])
  fourth_layer_code = encoder4.predict(third_layer_code)
  # Setting the weights of the deep autoencoder
  deep_autoencoder.layers[1].set_weights(encoder1.layers[-1].get_weights()) # first dense layer

  deep_autoencoder.layers[2].set_weights(encoder2.layers[-1].get_weights()) # second dense layer

  deep_autoencoder.layers[3].set_weights(encoder3.layers[-1].get_weights()) # thrird dense layer

  deep_autoencoder.layers[4].set_weights(encoder4.layers[-1].get_weights()) # fourth dense layer

  # deep_autoencoder.layers[5].set_weights(autoencoder4.layers[-1].get_weights()) # fourth dense layer

  deep_autoencoder.layers[6].set_weights(autoencoder3.layers[-1].get_weights()) # first decoder
  deep_autoencoder.layers[7].set_weights(autoencoder2.layers[-1].get_weights()) # second decoder
  deep_autoencoder.layers[8].set_weights(autoencoder1.layers[-1].get_weights()) # third decoder

  deep_autoencoder.fit(train_x, train_x,
                epochs = eph,
                validation_split = 0.0020,
                shuffle = False,
                verbose =0,
                callbacks=[keras.callbacks.EarlyStopping(patience=500, restore_best_weights=True)])

  sae_fingerprint = []

  for fingerprint in train_x:
    fingerprint = fingerprint.reshape(1, -1)
    sae_train_x = deep_autoencoder.predict(fingerprint, verbose = 0)
    sae_train_x = np.array(sae_train_x)
    rounded_data = np.round(sae_train_x[0], 2)  # Round to 2 decimal places
    sae_fingerprint.append(rounded_data)

  return np.array(sae_fingerprint), train_y


In [9]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import numpy as np

# Generate augmented data using GAN

def build_generator(noise_dim, output_dim):
    model = models.Sequential()
    model.add(layers.Dense(noise_dim, input_dim=noise_dim, activation='relu'))
    # model.add(layers.Dense(64, activation='tanh'))
    model.add(layers.Dense(int(noise_dim/4), activation='relu'))
    model.add(layers.Dense(output_dim, activation='sigmoid'))
    return model

# Discriminator Network
def build_discriminator(input_dim):
    model = models.Sequential()
    model.add(layers.Dense(128, input_dim=input_dim, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    # model.add(layers.Dense(32, activation='tanh'))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

def train_gan(train_x, train_y, noise_dim, batch_size, epochs):
    input_dim = train_x.shape[1]
    generator = build_generator(noise_dim, input_dim)
    discriminator = build_discriminator(input_dim)

    # Compile Discriminator
    discriminator.compile(optimizer=optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

    # Combined model (Generator + Discriminator)
    discriminator.trainable = False
    gan_input = layers.Input(shape=(noise_dim,))
    generated_data = generator(gan_input)
    gan_output = discriminator(generated_data)
    gan = models.Model(gan_input, gan_output)
    gan.compile(optimizer=optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy')

    # Training the GAN
    print("\n Training GAN... \n")
    for epoch in range(epochs):
        for _ in range(len(train_x) // batch_size):
            # Generate fake data
            noise = np.random.normal(0, 1, (batch_size, noise_dim))
            generated_data = generator.predict(noise, verbose=0)

            # Select a random batch of real data
            idx = np.random.randint(0, train_x.shape[0], batch_size)
            real_data = train_x[idx]

            # Labels for real and fake data
            real_labels = np.ones((batch_size, 1))
            fake_labels = np.zeros((batch_size, 1))

            # Train the Discriminator
            d_loss_real = discriminator.train_on_batch(real_data, real_labels)
            d_loss_fake = discriminator.train_on_batch(generated_data, fake_labels)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # Train the Generator
            noise = np.random.normal(0, 1, (batch_size, noise_dim))
            valid_y = np.ones((batch_size, 1))
            g_loss = gan.train_on_batch(noise, valid_y)

        if epoch :
            print(f'Epoch {epoch}, D Loss: {d_loss[0]:.4f}, D Acc.: {100*d_loss[1]:.2f}%')

    # Generate augmented data
    gan_x = []
    gan_y = []
    for i in range(len(train_x)):
        noise = np.random.normal(0, 1, (1, noise_dim))
        generated_data = generator.predict(noise, verbose=0)
        rounded_data = np.round(generated_data[0], 2)  # Round to 2 decimal places
        gan_x.append(rounded_data)
        gan_y.append(train_y[i])  # Assuming RP class remains the same

    sorted_indices = np.argsort(train_y)
    gan_x = np.array(gan_x)[sorted_indices]
    gan_y = np.array(gan_y)[sorted_indices]
    return gan_x, gan_y

# Example usage

# noise_dim = 100
# batch_size = 64
# epochs = 1000
# gan_x, gan_y = train_gan(train_x, train_y, noise_dim, batch_size, epochs)


In [10]:
# Functions Below :

train_dev = ['OP3']

dev = ['BLU','HTC','LG','MOTO','OP3','S7']

floorplan = ['engr0']


for train in train_dev:
    for flp in floorplan:
        print(f'\n Train dev -> {train}   flp -> {flp} \n')
        # Initial CI only !!!
        ci_val = 0
        train_x, train_y, train_aps = temp_train_data(train, flp, ci_val)
        sorted_indices = np.argsort(train_y)
        train_x = train_x[sorted_indices]
        train_y = train_y[sorted_indices]

# Generate augmented data using GAN
gan_x, gan_y = train_gan(train_x, train_y, noise_dim=int(train_x.shape[-1]), batch_size=int(train_x.shape[0]), epochs=100)

# Generate augmented data using Stacked SAE
stacked_sae_x, stacked_sae_y = stacked_sae(int(train_x.shape[-1]), train_x, train_y, 900)

# Generate augmented data using SAE

# Generate augmented data using FGSM

# Generate augmented data using PGD

# Generate augmented data using MIM

print('Original RSS')
print(train_x)
print("\n ---------------- \n")
print('GAN RSS')
print(gan_x)
print("\n ---------------- \n")
print('Stacked SAE RSS')
print(stacked_sae_x)








 Train dev -> OP3   flp -> engr0 


 Training GAN... 



/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


Epoch 1, D Loss: 0.7732, D Acc.: 53.07%
Epoch 2, D Loss: 0.7878, D Acc.: 50.69%
Epoch 3, D Loss: 0.7999, D Acc.: 49.36%
Epoch 4, D Loss: 0.8107, D Acc.: 48.31%
Epoch 5, D Loss: 0.8209, D Acc.: 47.68%
Epoch 6, D Loss: 0.8309, D Acc.: 47.36%
Epoch 7, D Loss: 0.8414, D Acc.: 47.18%
Epoch 8, D Loss: 0.8522, D Acc.: 46.92%
Epoch 9, D Loss: 0.8632, D Acc.: 46.69%
Epoch 10, D Loss: 0.8749, D Acc.: 46.59%
Epoch 11, D Loss: 0.8870, D Acc.: 46.41%
Epoch 12, D Loss: 0.8992, D Acc.: 46.37%
Epoch 13, D Loss: 0.9118, D Acc.: 46.24%
Epoch 14, D Loss: 0.9248, D Acc.: 46.19%
Epoch 15, D Loss: 0.9383, D Acc.: 46.22%
Epoch 16, D Loss: 0.9520, D Acc.: 46.17%
Epoch 17, D Loss: 0.9659, D Acc.: 46.15%
Epoch 18, D Loss: 0.9802, D Acc.: 46.14%
Epoch 19, D Loss: 0.9945, D Acc.: 46.12%
Epoch 20, D Loss: 1.0087, D Acc.: 46.09%
Epoch 21, D Loss: 1.0229, D Acc.: 46.03%
Epoch 22, D Loss: 1.0373, D Acc.: 45.98%
Epoch 23, D Loss: 1.0517, D Acc.: 45.92%
Epoch 24, D Loss: 1.0659, D Acc.: 45.90%
Epoch 25, D Loss: 1.0800,

In [11]:
''' Author - Danish Gufran
           - Jadon Johnson
           - Isaac Blair
           - Sudeep Pasricha

EPIC Lab : Colorado State University, Fort Collins

<Danish.Gufran@colostate.edu>
<Jadon.Johnson@colostate.edu>
<Isaac.Blair@colostate.edu>
<Sudeep@colostate.edu>

Title - Data Augmentation Methods for Wi-Fi RSS Fingerprinting Based Indoor Localization.

Citation :

'''

' Author - Danish Gufran\n           - Jadon Johnson\n           - Isaac Blair\n           - Sudeep Pasricha\n<Danish.Gufran@colostate.edu>\n<Jadon.Johnson@colostate.edu>\n<isaac.blair@colostate.edu>\n<sudeep@colostate.edu>\n\nTitle - Data Augmentation Methods for Wi-Fi RSS Fingerprinting Based Indoor Localization.\n\nCitation : \n\n'